In [20]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo

# 1. Extract

## Windows Users

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#load store page into browser, browser html output to html variable
url = 'https://store.steampowered.com/stats'
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; with browser html output
soup = BeautifulSoup(html)

In [6]:
# find the link on webpage to extend the games list to top 100 games
browser.find_link_by_text('View all of the top 100 most-played games').click()

C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:501: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [7]:
# read the new updated table on the webpage via browser into html
html = browser.html
soup = BeautifulSoup(html)
#print(soup)

In [8]:
# see what tables were pulled into pandas
tables = pd.read_html(html)
tables

[    0                      1
 0 NaN  Steam users logged in,
                          0         1         2   3
 0                      NaN   current      peak NaN
 1  Concurrent Steam Users:  15543931  23196994 NaN,
                    0           1   2                                     3   4
 0    Current Players  Peak Today NaN                                  Game NaN
 1                NaN         NaN NaN                                   NaN NaN
 2             418986      935172 NaN      Counter-Strike: Global Offensive NaN
 3             268722      589954 NaN                                Dota 2 NaN
 4             120075      135954 NaN                             Destiny 2 NaN
 ..               ...         ...  ..                                   ...  ..
 97              4718        6283 NaN                    Deep Rock Galactic NaN
 98              4543        5777 NaN                               Aim Lab NaN
 99              4502        9015 NaN              Assassin's 

# 2. Transform

In [9]:
# We want the 2nd table, load into games_df
games_df = tables[2]
games_df

,0,1,2,3,4
0,Current Players,Peak Today,NaN,Game,NaN
1,NaN,NaN,NaN,NaN,NaN
2,418986,935172,NaN,Counter-Strike: Global Offensive,NaN
3,268722,589954,NaN,Dota 2,NaN
4,120075,135954,NaN,Destiny 2,NaN
...,...,...,...,...,...
97,4718,6283,NaN,Deep Rock Galactic,NaN
98,4543,5777,NaN,Aim Lab,NaN
99,4502,9015,NaN,Assassin's Creed Odyssey,NaN
100,4501,7992,NaN,Total War: ROME II - Emperor Edition,NaN


In [10]:
#start cleaning table, get rid of garbage columns
del games_df[2]
del games_df[4]
games_df

,0,1,3
0,Current Players,Peak Today,Game
1,NaN,NaN,NaN
2,418986,935172,Counter-Strike: Global Offensive
3,268722,589954,Dota 2
4,120075,135954,Destiny 2
...,...,...,...
97,4718,6283,Deep Rock Galactic
98,4543,5777,Aim Lab
99,4502,9015,Assassin's Creed Odyssey
100,4501,7992,Total War: ROME II - Emperor Edition


In [11]:
# drop NaN row
games_df = games_df.dropna(how='all')
games_df

,0,1,3
0,Current Players,Peak Today,Game
2,418986,935172,Counter-Strike: Global Offensive
3,268722,589954,Dota 2
4,120075,135954,Destiny 2
5,88102,172892,Among Us
...,...,...,...
97,4718,6283,Deep Rock Galactic
98,4543,5777,Aim Lab
99,4502,9015,Assassin's Creed Odyssey
100,4501,7992,Total War: ROME II - Emperor Edition


In [12]:
# rename columns, get rid of garbage rows, reset index

games_df = games_df.rename(columns={0:"Current Players", 1:"Peak Today", 3:"Game"})
games_df = games_df.drop(games_df.index[0])
games_df = games_df.reset_index(drop=True)
games_df

,Current Players,Peak Today,Game
0,418986,935172,Counter-Strike: Global Offensive
1,268722,589954,Dota 2
2,120075,135954,Destiny 2
3,88102,172892,Among Us
4,76839,80734,Team Fortress 2
...,...,...,...
95,4718,6283,Deep Rock Galactic
96,4543,5777,Aim Lab
97,4502,9015,Assassin's Creed Odyssey
98,4501,7992,Total War: ROME II - Emperor Edition


In [13]:
browser.quit()

In [14]:
# Remove leading space from game in order to join with other data source

games_df['Game'] = games_df['Game'].str.lstrip()
games_df

,Current Players,Peak Today,Game
0,418986,935172,Counter-Strike: Global Offensive
1,268722,589954,Dota 2
2,120075,135954,Destiny 2
3,88102,172892,Among Us
4,76839,80734,Team Fortress 2
...,...,...,...
95,4718,6283,Deep Rock Galactic
96,4543,5777,Aim Lab
97,4502,9015,Assassin's Creed Odyssey
98,4501,7992,Total War: ROME II - Emperor Edition


In [15]:
# Write out file to cvs
games_df.to_csv('top_100_steam.csv')

In [24]:
# Read the Kaggle cvs datafile into pandas and filter columns 
steam_catalog = pd.read_csv('steam_games.csv').rename(columns={"name": "Game"})
game_info_df = steam_catalog[['Game', 'all_reviews', 'release_date', 'developer','popular_tags', 'game_details', 'achievements', 'genre', 'original_price', 'discount_price']]
game_info_df.head()

,Game,all_reviews,release_date,developer,popular_tags,game_details,achievements,genre,original_price,discount_price
0,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...",12-May-16,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...",54.0,Action,$19.99,$14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...",21-Dec-17,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats",37.0,"Action,Adventure,Massively Multiplayer",$29.99,NaN
2,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...",24-Apr-18,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...",128.0,"Action,Adventure,Strategy",$39.99,NaN
3,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...",13-Dec-18,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...",NaN,"Action,Adventure,Massively Multiplayer",$44.99,NaN
4,EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...",6-May-03,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",Free,NaN


In [25]:
# join/merge the two data sources
merge_df = pd.merge(games_df, game_info_df, on="Game", how="left").fillna('')
merge_df.head()

,Current Players,Peak Today,Game,all_reviews,release_date,developer,popular_tags,game_details,achievements,genre,original_price,discount_price
0,418986,935172,Counter-Strike: Global Offensive,"Very Positive,(3,094,223),- 86% of the 3,094,2...",21-Aug-12,"Valve,Hidden Path Entertainment","FPS,Shooter,Multiplayer,Competitive,Action,Tea...","Multi-player,Steam Achievements,Full controlle...",167,"Action,Free to Play",Free to Play,$65.70
1,268722,589954,Dota 2,"Very Positive,(1,015,621),- 85% of the 1,015,6...",9-Jul-13,Valve,"Free to Play,MOBA,Multiplayer,Strategy,e-sport...","Multi-player,Co-op,Steam Trading Cards,Steam W...",,"Action,Free to Play,Strategy",Free to Play,$65.70
2,120075,135954,Destiny 2,,17-Sep-19,Bungie,"FPS,Multiplayer,Sci-fi,Great Soundtrack,MMORPG...","Single-player,Online Multi-Player,Online Co-op...",,"Action,Adventure",$34.99,
3,88102,172892,Among Us,"Very Positive,(106),- 84% of the 106 user revi...",16-Nov-18,Innersloth,"Casual,Multiplayer,Local Multiplayer,Space,Onl...","Multi-player,Online Multi-Player,Local Multi-P...",,Casual,$4.99,
4,76839,80734,Team Fortress 2,"Very Positive,(553,458),- 93% of the 553,458 u...",10-Oct-07,Valve,"Free to Play,Multiplayer,FPS,Shooter,Action,Cl...","Multi-player,Cross-Platform Multiplayer,Steam ...",520,"Action,Free to Play",Free to Play,$65.70


In [ ]:
# print the merged_df to a csv file.  
games_df.to_csv('merge.csv')

# information was missing in 26 of the games, as they recently launched.  
# data was manually extracted from Steam's website to fill in missing cells.

In [63]:
# read the updated csv file to a pandas DataFrame. 
# This is the final compiled data file to be put into the database. 
steam_100 = pd.read_csv('merge_edits.csv', encoding= 'unicode_escape').fillna('')
del steam_100['Unnamed: 0']
steam_final_df = steam_100.sort_values(by=['current_player'], ascending=False).reset_index(drop=True)
steam_final_df


,current_player,peak_today,game,all_reviews,release_date,developer,popular_tags,game_details,achievements,genre,original_price,discount_price
0,802725,1040051,Counter-Strike: Global Offensive,"Very Positive,(3,094,223),- 86% of the 3,094,2...",21-Aug-12,"Valve,Hidden Path Entertainment","FPS,Shooter,Multiplayer,Competitive,Action,Tea...","Multi-player,Steam Achievements,Full controlle...",167,"Action,Free to Play",Free to Play,$65.70
1,431365,678801,Dota 2,"Very Positive,(1,015,621),- 85% of the 1,015,6...",9-Jul-13,Valve,"Free to Play,MOBA,Multiplayer,Strategy,e-sport...","Multi-player,Co-op,Steam Trading Cards,Steam W...",,"Action,Free to Play,Strategy",Free to Play,$65.70
2,247776,250338,Among Us,"Very Positive,(106),- 84% of the 106 user revi...",16-Nov-18,Innersloth,"Casual,Multiplayer,Local Multiplayer,Space,Onl...","Multi-player,Online Multi-Player,Local Multi-P...",,Casual,$4.99,
3,193621,196805,Destiny 2,"Mostly Positive (17,560)",17-Sep-19,Bungie,"FPS,Multiplayer,Sci-fi,Great Soundtrack,MMORPG...","Single-player,Online Multi-Player,Online Co-op...",,"Action,Adventure",$34.99,
4,117465,121519,Rocket League,"Very Positive,(188,275),*,- 91% of the 188,275...",7-Jul-15,"Psyonix, Inc.","Multiplayer,Soccer,Competitive,Sports,Racing,T...","Single-player,Multi-player,Co-op,Shared/Split ...",88,"Action,Indie,Racing,Sports",$19.99,
...,...,...,...,...,...,...,...,...,...,...,...,...
95,7828,8417,Geometry Dash,"Very Positive,(51,251),- 94% of the 51,251 use...",22-Dec-14,RobTop Games,"Difficult,Great Soundtrack,Music,Level Editor,...","Single-player,Steam Achievements,Steam Trading...",120,"Action,Indie",$3.99,
96,7242,11550,eFootball PES 2021 SEASON UPDATE,"Mostly Positive (1,382)",15-Sep-20,Konami Digital Entertainment,"Sports, Realistic, eSports, Co-op, Multiplayer...",Multi-player,,Sports,$29.99,
97,7072,9346,F1 2020,"Very Positive (8,004)",9-Jul-20,Codemasters,"Racing, Sports, Simulation, Automobile Sim, Mu...",Single-player,50,"Racing, Simulation, Sports",$59.99,$35.99
98,7045,7894,The Binding of Isaac: Rebirth,"Overwhelmingly Positive,(63,149),- 97% of the ...",4-Nov-14,"Nicalis, Inc.","Rogue-like,Indie,Replay Value,Difficult,Pixel ...","Single-player,Shared/Split Screen,Steam Achiev...",403,Action,$14.99,$35.97


# 3. Load

In [64]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [65]:
# drop existing db to prevent duplicate records
db.games.drop()


In [66]:
# Define database and collection
db = client.steam_games_db
collection = db.games

In [67]:
# Dictionary to be inserted as a MongoDB document
data_dict = steam_final_df.to_dict("records")

# Insert collection
collection.insert_many(data_dict)


In [68]:
# Display MongoDB records within the collection

games = db.games.find()

for game in games:
    print(game)

{'_id': ObjectId('5fbc6e965f1c419d4230b927'), 'current_player': 802725, 'peak_today': 1040051, 'game': 'Counter-Strike: Global Offensive', 'all_reviews': 'Very Positive,(3,094,223),- 86% of the 3,094,223 user reviews for this game are positive.', 'release_date': '21-Aug-12', 'developer': 'Valve,Hidden Path Entertainment', 'popular_tags': 'FPS,Shooter,Multiplayer,Competitive,Action,Team-Based,e-sports,Tactical,First-Person,PvP,Online Co-Op,Co-op,Strategy,Military,War,Difficult,Trading,Fast-Paced,Realistic,Moddable', 'game_details': 'Multi-player,Steam Achievements,Full controller support,Steam Trading Cards,Steam Workshop,In-App Purchases,Valve Anti-Cheat enabled,Stats', 'achievements': 167.0, 'genre': 'Action,Free to Play', 'original_price': 'Free to Play', 'discount_price': '$65.70 '}
{'_id': ObjectId('5fbc6e965f1c419d4230b928'), 'current_player': 431365, 'peak_today': 678801, 'game': 'Dota 2', 'all_reviews': 'Very Positive,(1,015,621),- 85% of the 1,015,621 user reviews for this game

In [71]:
# confirms we have successfully loaded all 100 records into the database. 
db.games.count()

C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


100